In [16]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document

import os

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = 'you code'

# Initialize components
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(embedding_function=embeddings)
llm = ChatOpenAI(temperature=0)

# Populate the vector store with some documents
documents = [
    Document(page_content = "Renewable energy sources like wind, solar, and hydroelectric power have various environmental impacts. These impacts can be both positive and negative. For instance, wind turbines can affect bird migration patterns, while solar farms can alter local ecosystems."),
    Document(page_content = "Renewable energy reduces reliance on fossil fuels, decreasing greenhouse gas emissions and mitigating climate change. However, the production and disposal of renewable energy infrastructure can result in environmental challenges, including resource depletion and waste generation."),
    Document(page_content = "The transition to renewable energy is essential for sustainable development, providing long-term benefits for the environment. Nonetheless, careful planning and management are required to minimize adverse effects on wildlife, water resources, and land use."),
]

# Add documents to the vector store
vectorstore.add_documents(documents)

# Create a prompt for generating hypothetical documents
hyde_prompt = PromptTemplate(
    input_variables=["question"],
    template="Given the question '{question}', generate a hypothetical passage that would answer this question:"
)

# Create an LLMChain for generating hypothetical documents
hyde_chain = LLMChain(llm=llm, prompt=hyde_prompt)

# Function to perform RAG HyDE
def rag_hyde(query):
    # Generate hypothetical document
    hypothetical_doc = hyde_chain.run(question=query)
    #print(f"Hypothetical Document: {hypothetical_doc}")
    
    # Embed and search using the hypothetical document
    hypothetical_embedding = embeddings.embed_query(hypothetical_doc)
    #print(f"Hypothetical Embedding: {hypothetical_embedding}")
    
    results = vectorstore.similarity_search_by_vector(hypothetical_embedding, k=5)
    #print(f"Search Results: {results}")
    
    return results

# Perform a query
query = "What are the environmental impacts of renewable energy?"
hyde_results = rag_hyde(query)

for doc in hyde_results:
    print(f"Retrieved document: {doc.page_content[:100]}...")
    print()


Retrieved document: Renewable energy sources like wind, solar, and hydroelectric power have various environmental impact...

Retrieved document: Renewable energy sources like wind, solar, and hydroelectric power have various environmental impact...

Retrieved document: Renewable energy sources like wind, solar, and hydroelectric power have various environmental impact...

Retrieved document: Renewable energy reduces reliance on fossil fuels, decreasing greenhouse gas emissions and mitigatin...

Retrieved document: Renewable energy reduces reliance on fossil fuels, decreasing greenhouse gas emissions and mitigatin...

